# 对概念的理解和举例


### Character (字符)
- **定义**: 人类可读的文本单位
- **示例**: `'h'`, `'e'`, `'中'`, `'文'`, `'😀'`
- **Python 类型**: `str`
- **特点**: 
  - Unicode 字符数量巨大 (超过 100 万种)
  - 不同字符的 Unicode 码点不同
  - 字符数 ≠ 字节数

### Byte (字节)
- **定义**: 计算机存储的基本单位
- **范围**: 0-255 (2^8 = 256 种可能)
- **Python 类型**: `bytes`
- **特点**:
  - 固定范围: 0-255
  - 1 字节 = 8 位 (bit)
  - 所有文本都可以转换为字节序列

### Unicode
- **定义**：字符编码标准，为每个字符分配唯一的码点（标准）
- **Unicode 码点 (Code Point)**: 每个字符都有一个唯一的 Unicode 码点，通常用 `U+` 前缀表示（十六进制），例如: `U+0068`, `U+4E2D`, `U+1F600`
- **示例**: 
```python
ord('h')    # 104 (U+0068) 
ord('中')   # 20013 (U+4E2D)
```
- **Python 函数**: ord(char)


### UTF-8 编码

- **定义**: 将 Unicode 码点编码成字节序列的方式(计算机需要字节序列来存储)


- **示例**:
```python
'h'.encode('utf-8')    # b'h' (1 字节)
'中'.encode('utf-8')   # b'\xe4\xb8\xad' (3 字节)
'😀'.encode('utf-8')   # b'\xf0\x9f\x98\x80' (4 字节)
```

- **Python 方法**: char.encode('utf-8')

## 字符与字节的关系

```
Character (字符)
    ↓
Unicode 码点 (数字标识符)
    ↓
UTF-8 编码 (字节序列)
```

### 示例: `'中'`

```
字符: '中'
    ↓
Unicode 码点: 20013 (U+4E2D)
    ↓
UTF-8 编码: b'\xe4\xb8\xad' (3 字节)
    ↓
字节值: [228, 184, 173]
```

概念容易混淆：
- Unicode 不是字符，Unicode 是字符编码标准，为字符分配码点，是字符的数字表示。
- Unicode 和 UTF-8 不是一回事，Unicode 是标准，（定义字符和码点），UTF-8 是编码方式（将码点编码成字节）。



### 字节级别的优势

1. **初始词汇表大小固定**
   - 字符级别: 需要处理所有 Unicode 字符 (100万+)
   - 字节级别: 只需要 256 个初始 token (0-255)

2. **覆盖性**
   - 字符级别: 无法处理训练时未见过的字符
   - 字节级别: 可以处理任何 Unicode 字符

3. **统一性**
   - 字符级别: 不同语言的字符集差异很大
   - 字节级别: 所有文本都可以转换为字节序列

4. **简单性**
   - 字节范围固定 (0-255)
   - 可以学习常见字节组合的模式




# 分词器训练出现的概念


### 1. vocab (词汇表)

**类型**: `dict[int, bytes]`

**含义**: 从 token ID (整数) 映射到 token 的字节表示

**结构**:
```python
vocab = {
    0: b'\x00',        # token ID 0 -> 字节 0
    1: b'\x01',        # token ID 1 -> 字节 1
    ...
    104: b'h',         # token ID 104 -> 'h' 字节
    256: b'he',        # token ID 256 -> 'he' (BPE合并后的token)
    257: b'll',        # token ID 257 -> 'll' (BPE合并后的token)
    258: b'hello',     # token ID 258 -> 'hello' (BPE合并后的token)
}
```

**序列化后** (用于保存到 JSON):
```python
vocab_serialized = {
    'Ā': 0,           # 字节 0 对应的 unicode 字符
    'ā': 1,           # 字节 1 对应的 unicode 字符
    'Ġ': 32,          # 空格字节对应的 unicode 字符
    'h': 104,         # 'h' 字节
    'he': 256,        # 'he' token
    'll': 257,        # 'll' token
    'hello': 258,     # 'hello' token
}
```

---

### 2. merges (合并规则)

**类型**: `list[tuple[bytes, bytes]]`

**含义**: BPE训练过程中，按顺序记录的合并操作。每个元组 `(token1, token2)` 表示将 `token1` 和 `token2` 合并成一个新的 token。

**结构**:
```python
merges = [
    (b'h', b'e'),         # 第1次合并: h + e -> he
    (b'l', b'l'),         # 第2次合并: l + l -> ll
    (b'he', b'll'),       # 第3次合并: he + ll -> hell
    (b'hell', b'o'),      # 第4次合并: hell + o -> hello
]
```

**序列化后** (用于保存到文本文件):
```python
merges_serialized = [
    'h e',           # 第1次合并
    'l l',           # 第2次合并
    'he ll',         # 第3次合并
    'hell o',        # 第4次合并
]
```

---

### 3. 为什么需要序列化
序列化是指

JSON 不支持 `bytes` 类型！
---

## 🔄 BPE 合并过程示例

假设我们要对单词 **"hello"** 进行 BPE tokenization：

### 初始状态
```
原始文本: "hello"
UTF-8 编码: b'hello'
字节值: [104, 101, 108, 108, 111]
初始 tokens: [b'h', b'e', b'l', b'l', b'o']
```

### 应用 merges (按顺序)

**步骤 1**: 合并 `b'h'` + `b'e'`
```
合并前: [b'h', b'e', b'l', b'l', b'o']
合并后: [b'he', b'l', b'l', b'o']
```

**步骤 2**: 合并 `b'l'` + `b'l'`
```
合并前: [b'he', b'l', b'l', b'o']
合并后: [b'he', b'll', b'o']
```

**步骤 3**: 合并 `b'he'` + `b'll'`
```
合并前: [b'he', b'll', b'o']
合并后: [b'hell', b'o']
```

**步骤 4**: 合并 `b'hell'` + `b'o'`
```
合并前: [b'hell', b'o']
合并后: [b'hello']
```

### 最终结果
```
最终 tokens: [b'hello']
最终 token ID: 258 (假设 vocab[258] = b'hello')
```

---

## 💾 文件保存格式

### vocab.json
```json
{
  "Ā": 0,
  "ā": 1,
  "Ġ": 32,
  "h": 104,
  "he": 256,
  "ll": 257,
  "hello": 258
}
```

### merges.txt
```
h e
l l
he ll
hell o
```


# merge 里面的pre_token_counts和 merge_tables 
## 初始状态
```python
pre_token_counts = { (b'h', b'e', b'l', b'l', b'o'): 2,  # "hello" 出现2次   
(b'w', b'o', b'r', b'l', b'd'): 1,  #"world" 出现1次
}
merge_tables = {    (b'h', b'e'): 2,  # h-e 出现2次    
(b'e', b'l'): 2,  # e-l 出现2次    
(b'l', b'l'): 2,  # l-l 出现2次 ← 最高频！    
(b'l', b'o'): 2,  # l-o 出现2次    
(b'w', b'o'): 1,  # w-o 出现1次    
(b'o', b'r'): 1,  # o-r 出现1次    
(b'r', b'l'): 1,  # r-l 出现1次    
(b'l', b'd'): 1,  # l-d 出现1次}
```

## 第1步：找到最高频 pair
(b'l', b'l') 频率最高，合并为 b'll'
第2步：先更新 pre_token_counts

## 第二步：先更新 pre_token_counts
```python
# 更新前
pre_token_counts = { (b'h', b'e', b'l', b'l', b'o'): 2,   
 (b'w', b'o', b'r', b'l', b'd'): 1,} 
 
 
 # 更新后（将 l-l 替换为 ll）
 pre_token_counts = {(b'h', b'e', b'll', b'o'): 2,   # hello → he + ll + o    
 (b'w', b'o', b'r', b'l', b'd'): 1, #world 中没有 l-l，不变
 }
```
## 第3步：基于新的 pre_token_counts 更新 merge_tables

```python

merge_tables = {
(b'h', b'e'): 2, # h-e 仍然2次    
(b'e', b'll'): 2, # e-ll 出现2次（新增）
(b'll', b'o'): 2,  # ll-o 出现2次（新增）    
(b'w', b'o'): 1,      # w-o 仍然1次  
(b'o', b'r'): 1,      # o-r 仍然1次  
(b'r', b'l'): 1,      # r-l 仍然1次  
(b'l', b'd'): 1,      # l-d 仍然1次    
#(b'l', b'l') 已经不存在了 

}
```


# Profile 部分

step 1: add the special token to the vocabulary

In [63]:
vocab = {}

special_token = "<|endoftext|>"
vocab.update({special_token: 0})


step 2: add 256 byte values to the vocabulary

In [64]:
for i in range(256):
    char = chr(i)
    vocab.update({char: i + 1})

In [65]:
vocab

{'<|endoftext|>': 0,
 '\x00': 1,
 '\x01': 2,
 '\x02': 3,
 '\x03': 4,
 '\x04': 5,
 '\x05': 6,
 '\x06': 7,
 '\x07': 8,
 '\x08': 9,
 '\t': 10,
 '\n': 11,
 '\x0b': 12,
 '\x0c': 13,
 '\r': 14,
 '\x0e': 15,
 '\x0f': 16,
 '\x10': 17,
 '\x11': 18,
 '\x12': 19,
 '\x13': 20,
 '\x14': 21,
 '\x15': 22,
 '\x16': 23,
 '\x17': 24,
 '\x18': 25,
 '\x19': 26,
 '\x1a': 27,
 '\x1b': 28,
 '\x1c': 29,
 '\x1d': 30,
 '\x1e': 31,
 '\x1f': 32,
 ' ': 33,
 '!': 34,
 '"': 35,
 '#': 36,
 '$': 37,
 '%': 38,
 '&': 39,
 "'": 40,
 '(': 41,
 ')': 42,
 '*': 43,
 '+': 44,
 ',': 45,
 '-': 46,
 '.': 47,
 '/': 48,
 '0': 49,
 '1': 50,
 '2': 51,
 '3': 52,
 '4': 53,
 '5': 54,
 '6': 55,
 '7': 56,
 '8': 57,
 '9': 58,
 ':': 59,
 ';': 60,
 '<': 61,
 '=': 62,
 '>': 63,
 '?': 64,
 '@': 65,
 'A': 66,
 'B': 67,
 'C': 68,
 'D': 69,
 'E': 70,
 'F': 71,
 'G': 72,
 'H': 73,
 'I': 74,
 'J': 75,
 'K': 76,
 'L': 77,
 'M': 78,
 'N': 79,
 'O': 80,
 'P': 81,
 'Q': 82,
 'R': 83,
 'S': 84,
 'T': 85,
 'U': 86,
 'V': 87,
 'W': 88,
 'X': 89,
 'Y': 90

step 3: pretokenization

In [66]:
text = """low low low low low
lower lower widest widest widest
newest newest newest newest newest newest"""


words = text.split()
print(words)
from collections import Counter
word_counts = Counter(words)
frequency_table = dict(word_counts)

frequency_table = {tuple(word): count for word, count in frequency_table.items()}
frequency_table

['low', 'low', 'low', 'low', 'low', 'lower', 'lower', 'widest', 'widest', 'widest', 'newest', 'newest', 'newest', 'newest', 'newest', 'newest']


{('l', 'o', 'w'): 5,
 ('l', 'o', 'w', 'e', 'r'): 2,
 ('w', 'i', 'd', 'e', 's', 't'): 3,
 ('n', 'e', 'w', 'e', 's', 't'): 6}

step4: merge

In [42]:
#  count the frequency of all adjacent character pairs

merge_tables = {}
for word, count in frequency_table.items():
   
    for i in range(len(word)-1):
     
        char_pair = word[i:i+2]
        if char_pair not in merge_tables:
            merge_tables[char_pair] = count
        else:
            merge_tables[char_pair] += count
    
merge_tables


{('l', 'o'): 7,
 ('o', 'w'): 7,
 ('w', 'e'): 8,
 ('e', 'r'): 2,
 ('w', 'i'): 3,
 ('i', 'd'): 3,
 ('d', 'e'): 3,
 ('e', 's'): 9,
 ('s', 't'): 9,
 ('n', 'e'): 6,
 ('e', 'w'): 6}

In [47]:
# merge the most frequent pair
most_frequent_pair = max(merge_tables, key=merge_tables.get)
print("Most frequent pair:", most_frequent_pair, "with count:", merge_tables[most_frequent_pair])

# update the frequency table by merging the most frequent pair
new_frequency_table = {}
for word, count in frequency_table.items():
    new_word = []
    i = 0
    while i < len(word):
        if i < len(word) - 1 and word[i:i+2] == most_frequent_pair:
            new_word.append(''.join(most_frequent_pair))
            i += 2
        else:
            new_word.append(word[i])
            i += 1
    new_frequency_table[tuple(new_word)] = count


new_frequency_table

Most frequent pair: ('e', 's') with count: 9


{('l', 'o', 'w'): 5,
 ('l', 'o', 'w', 'e', 'r'): 2,
 ('w', 'i', 'd', 'es', 't'): 3,
 ('n', 'e', 'w', 'es', 't'): 6}

In [49]:
#  count the frequency of all adjacent character pairs
merge_tables = {}
for word, count in new_frequency_table.items():
    
     for i in range(len(word)-1):
      
          char_pair = word[i:i+2]
          if char_pair not in merge_tables:
                merge_tables[char_pair] = count
          else:
                merge_tables[char_pair] += count

merge_tables

{('l', 'o'): 7,
 ('o', 'w'): 7,
 ('w', 'e'): 2,
 ('e', 'r'): 2,
 ('w', 'i'): 3,
 ('i', 'd'): 3,
 ('d', 'es'): 3,
 ('es', 't'): 9,
 ('n', 'e'): 6,
 ('e', 'w'): 6,
 ('w', 'es'): 6}

In [50]:
# merge the most frequent pair
most_frequent_pair = max(merge_tables, key=merge_tables.get)
print("Most frequent pair:", most_frequent_pair, "with count:", merge_tables[most_frequent_pair])

# update the frequency table by merging the most frequent pair
frequency_table = {}
for word, count in new_frequency_table.items():
    new_word = []
    i = 0
    while i < len(word):
        if i < len(word) - 1 and word[i:i+2] == most_frequent_pair:
            new_word.append(''.join(most_frequent_pair))
            i += 2
        else:
            new_word.append(word[i])
            i += 1
    frequency_table[tuple(new_word)] = count


frequency_table

Most frequent pair: ('es', 't') with count: 9


{('l', 'o', 'w'): 5,
 ('l', 'o', 'w', 'e', 'r'): 2,
 ('w', 'i', 'd', 'est'): 3,
 ('n', 'e', 'w', 'est'): 6}

将merge这个过程写成loop

In [80]:
def merge(merge_counts):
    current_count = 1
    while current_count <= merge_counts:
        print("\nMerge iteration:", current_count)
        #  count the frequency of all adjacent character pairs
        merge_tables = {}
        for word, count in frequency_table.items():
            
             for i in range(len(word)-1):
              
                  char_pair = word[i:i+2]
                  if char_pair not in merge_tables:
                        merge_tables[char_pair] = count
                  else:
                        merge_tables[char_pair] += count

        # merge the most frequent pair
        most_frequent_pair = max(merge_tables, key=merge_tables.get)
        print("Most frequent pair:", most_frequent_pair, "with count:", merge_tables[most_frequent_pair])

        # update the frequency table by merging the most frequent pair
        new_frequency_table = {}
        for word, count in frequency_table.items():
            new_word = []
            i = 0
            while i < len(word):
                if i < len(word) - 1 and word[i:i+2] == most_frequent_pair:
                    new_word.append(''.join(most_frequent_pair))
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_frequency_table[tuple(new_word)] = count

        frequency_table.clear()
        frequency_table.update(new_frequency_table)
        current_count += 1 

    return frequency_table, merge_tables

In [81]:
frequency_table, merge_tables = merge(5)


Merge iteration: 1
Most frequent pair: ('lo', 'w') with count: 7

Merge iteration: 2
Most frequent pair: ('n', 'e') with count: 6

Merge iteration: 3
Most frequent pair: ('ne', 'w') with count: 6

Merge iteration: 4
Most frequent pair: ('new', 'est') with count: 6

Merge iteration: 5
Most frequent pair: ('w', 'i') with count: 3


In [82]:
merge_tables

{('low', 'e'): 2, ('e', 'r'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est'): 3}

In [83]:
frequency_table

{('low',): 5, ('low', 'e', 'r'): 2, ('wi', 'd', 'est'): 3, ('newest',): 6}

# training valid text result

================================================================================
Performance Profile (sorted by cumulative time - shows call chains)
================================================================================
         128736260 function calls (128735955 primitive calls) in 630.064 seconds

   Ordered by: cumulative time
   List reduced from 511 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.104    0.104  630.064  630.064 /root/projects/assignment1-basics/cs336_basics/train_bpe.py:233(bpe_tokenizer)
        1  527.712  527.712  629.170  629.170 /root/projects/assignment1-basics/cs336_basics/train_bpe.py:51(merge)
127764200/127764160   99.081    0.000   99.081    0.000 {built-in method builtins.len}
   297940    1.171    0.000    1.171    0.000 {method 'append' of 'list' objects}
   194278    0.787    0.000    0.787    0.000 {built-in method _heapq.heappop}
        1    0.004    0.004    0.747    0.747 /root/projects/assignment1-basics/cs336_basics/pretokenization_example.py:42(wrapper)
        1    0.000    0.000    0.742    0.742 /root/projects/assignment1-basics/cs336_basics/pretokenization_example.py:180(process_parallel)
        1    0.000    0.000    0.695    0.695 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/pool.py:738(__exit__)
        1    0.000    0.000    0.668    0.668 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/pool.py:654(terminate)
       21    0.000    0.000    0.654    0.031 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/util.py:272(__call__)
        1    0.000    0.000    0.654    0.654 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/pool.py:680(_terminate_pool)
        1    0.000    0.000    0.653    0.653 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/pool.py:671(_help_stuff_finish)
        1    0.002    0.002    0.653    0.653 {method 'acquire' of '_multiprocessing.SemLock' objects}
       40    0.000    0.000    0.651    0.016 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/connection.py:202(send)
      3/1    0.000    0.000    0.651    0.651 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/threading.py:1000(_bootstrap)
      3/1    0.000    0.000    0.651    0.651 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/threading.py:1027(_bootstrap_inner)
      3/1    0.000    0.000    0.650    0.650 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/threading.py:983(run)
        1    0.000    0.000    0.650    0.650 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/pool.py:527(_handle_tasks)
       45    0.000    0.000    0.650    0.014 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/connection.py:406(_send_bytes)
       45    0.000    0.000    0.650    0.014 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/connection.py:381(_send)



================================================================================
Performance Profile (sorted by internal time - shows most time-consuming functions)
================================================================================
         128736260 function calls (128735955 primitive calls) in 630.064 seconds

   Ordered by: internal time
   List reduced from 511 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1  527.712  527.712  629.170  629.170 /root/projects/assignment1-basics/cs336_basics/train_bpe.py:51(merge)
127764200/127764160   99.081    0.000   99.081    0.000 {built-in method builtins.len}
   297940    1.171    0.000    1.171    0.000 {method 'append' of 'list' objects}
   194278    0.787    0.000    0.787    0.000 {built-in method _heapq.heappop}
    46/42    0.556    0.012    0.557    0.013 {built-in method posix.read}
   217352    0.382    0.000    0.382    0.000 {built-in method _heapq.heappush}
        1    0.104    0.104  630.064  630.064 /root/projects/assignment1-basics/cs336_basics/train_bpe.py:233(bpe_tokenizer)
       23    0.054    0.002    0.054    0.002 {built-in method _pickle.loads}
       21    0.039    0.002    0.047    0.002 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/collections/__init__.py:673(update)
     9772    0.032    0.000    0.032    0.000 {method 'items' of 'dict' objects}
   103955    0.028    0.000    0.028    0.000 /root/projects/assignment1-basics/cs336_basics/train_bpe.py:253(<genexpr>)
    13113    0.015    0.000    0.015    0.000 {method 'encode' of 'str' objects}
      159    0.012    0.000    0.550    0.003 {method 'poll' of 'select.poll' objects}
    92860    0.008    0.000    0.008    0.000 {method 'get' of 'dict' objects}
      104    0.007    0.000    0.007    0.000 {built-in method builtins.print}
       18    0.005    0.000    0.005    0.000 {built-in method marshal.loads}
        1    0.004    0.004    0.747    0.747 /root/projects/assignment1-basics/cs336_basics/pretokenization_example.py:42(wrapper)
       20    0.004    0.000    0.004    0.000 {built-in method posix.fork}
     1797    0.004    0.000    0.012    0.000 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/selectors.py:340(register)
     1797    0.003    0.000    0.007    0.000 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/selectors.py:238(register)



================================================================================
Training Summary
================================================================================
Time taken: 10 minute(s), 30.08 seconds (630.08 seconds total)
Memory usage: 4.80 MB (peak: 18.83 MB)
================================================================================

Saved vocab to: /root/projects/assignment1-basics/data/tinystories-valid_vocab.json
Saved merges to: /root/projects/assignment1-basics/data/tinystories-valid_merges.txt

================================================================================
Longest Token Analysis
================================================================================
Longest token (serialized string): 'Ġaccomplishment'
Token length: 15 characters (in serialized form)
Token ID: 7338
Token bytes: b' accomplishment'
Token bytes length: 15 bytes
Decoded as UTF-8: ' accomplishment'
Does it make sense? Yes - contains readable text
================================================================================

# train train-text

并行处理 耗时: 65.74 秒
Pretokenization complete. Unique tokens: 59933
Converting tokens to byte sequences...
Conversion complete. Unique byte sequences: 59933
Initializing merge_tables with 59933 unique tokens...
Starting BPE merging: 9743 merges needed...
Progress: 100/9743 merges completed (1.0%)
Progress: 200/9743 merges completed (2.1%)
Progress: 300/9743 merges completed (3.1%)
Progress: 400/9743 merges completed (4.1%)
Progress: 500/9743 merges completed (5.1%)
Progress: 600/9743 merges completed (6.2%)
Progress: 700/9743 merges completed (7.2%)
Progress: 800/9743 merges completed (8.2%)
Progress: 900/9743 merges completed (9.2%)
Progress: 1000/9743 merges completed (10.3%)
Progress: 1100/9743 merges completed (11.3%)
Progress: 1200/9743 merges completed (12.3%)
Progress: 1300/9743 merges completed (13.3%)
Progress: 1400/9743 merges completed (14.4%)
Progress: 1500/9743 merges completed (15.4%)
Progress: 1600/9743 merges completed (16.4%)
Progress: 1700/9743 merges completed (17.4%)
Progress: 1800/9743 merges completed (18.5%)
Progress: 1900/9743 merges completed (19.5%)
Progress: 2000/9743 merges completed (20.5%)
Progress: 2100/9743 merges completed (21.6%)
Progress: 2200/9743 merges completed (22.6%)
Progress: 2300/9743 merges completed (23.6%)
Progress: 2400/9743 merges completed (24.6%)
Progress: 2500/9743 merges completed (25.7%)
Progress: 2600/9743 merges completed (26.7%)
Progress: 2700/9743 merges completed (27.7%)
Progress: 2800/9743 merges completed (28.7%)
Progress: 2900/9743 merges completed (29.8%)
Progress: 3000/9743 merges completed (30.8%)
Progress: 3100/9743 merges completed (31.8%)
Progress: 3200/9743 merges completed (32.8%)
Progress: 3300/9743 merges completed (33.9%)
Progress: 3400/9743 merges completed (34.9%)
Progress: 3500/9743 merges completed (35.9%)
Progress: 3600/9743 merges completed (36.9%)
Progress: 3700/9743 merges completed (38.0%)
Progress: 3800/9743 merges completed (39.0%)
Progress: 3900/9743 merges completed (40.0%)
Progress: 4000/9743 merges completed (41.1%)
Progress: 4100/9743 merges completed (42.1%)
Progress: 4200/9743 merges completed (43.1%)
Progress: 4300/9743 merges completed (44.1%)
Progress: 4400/9743 merges completed (45.2%)
Progress: 4500/9743 merges completed (46.2%)
Progress: 4600/9743 merges completed (47.2%)
Progress: 4700/9743 merges completed (48.2%)
Progress: 4800/9743 merges completed (49.3%)
Progress: 4900/9743 merges completed (50.3%)
Progress: 5000/9743 merges completed (51.3%)
Progress: 5100/9743 merges completed (52.3%)
Progress: 5200/9743 merges completed (53.4%)
Progress: 5300/9743 merges completed (54.4%)
Progress: 5400/9743 merges completed (55.4%)
Progress: 5500/9743 merges completed (56.5%)
Progress: 5600/9743 merges completed (57.5%)
Progress: 5700/9743 merges completed (58.5%)
Progress: 5800/9743 merges completed (59.5%)
Progress: 5900/9743 merges completed (60.6%)
Progress: 6000/9743 merges completed (61.6%)
Progress: 6100/9743 merges completed (62.6%)
Progress: 6200/9743 merges completed (63.6%)
Progress: 6300/9743 merges completed (64.7%)
Progress: 6400/9743 merges completed (65.7%)
Progress: 6500/9743 merges completed (66.7%)
Progress: 6600/9743 merges completed (67.7%)
Progress: 6700/9743 merges completed (68.8%)
Progress: 6800/9743 merges completed (69.8%)
Progress: 6900/9743 merges completed (70.8%)
Progress: 7000/9743 merges completed (71.8%)
Progress: 7100/9743 merges completed (72.9%)
Progress: 7200/9743 merges completed (73.9%)
Progress: 7300/9743 merges completed (74.9%)
Progress: 7400/9743 merges completed (76.0%)
Progress: 7500/9743 merges completed (77.0%)
Progress: 7600/9743 merges completed (78.0%)
Progress: 7700/9743 merges completed (79.0%)
Progress: 7800/9743 merges completed (80.1%)
Progress: 7900/9743 merges completed (81.1%)
Progress: 8000/9743 merges completed (82.1%)
Progress: 8100/9743 merges completed (83.1%)
Progress: 8200/9743 merges completed (84.2%)
Progress: 8300/9743 merges completed (85.2%)
Progress: 8400/9743 merges completed (86.2%)
Progress: 8500/9743 merges completed (87.2%)
Progress: 8600/9743 merges completed (88.3%)
Progress: 8700/9743 merges completed (89.3%)
Progress: 8800/9743 merges completed (90.3%)
Progress: 8900/9743 merges completed (91.3%)
Progress: 9000/9743 merges completed (92.4%)
Progress: 9100/9743 merges completed (93.4%)
Progress: 9200/9743 merges completed (94.4%)
Progress: 9300/9743 merges completed (95.5%)
Progress: 9400/9743 merges completed (96.5%)
Progress: 9500/9743 merges completed (97.5%)
Progress: 9600/9743 merges completed (98.5%)
Progress: 9700/9743 merges completed (99.6%)
Progress: 9743/9743 merges completed (100.0%)

================================================================================
Performance Profile (sorted by cumulative time - shows call chains)
================================================================================
         588197441 function calls (588197136 primitive calls) in 3624.529 seconds

   Ordered by: cumulative time
   List reduced from 511 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.929    0.929 3624.528 3624.528 /root/projects/assignment1-basics/cs336_basics/train_bpe.py:233(bpe_tokenizer)
        1 3091.468 3091.468 3557.664 3557.664 /root/projects/assignment1-basics/cs336_basics/train_bpe.py:51(merge)
583997919/583997879  455.537    0.000  455.537    0.000 {built-in method builtins.len}
        1    0.034    0.034   65.737   65.737 /root/projects/assignment1-basics/cs336_basics/pretokenization_example.py:42(wrapper)
        1    0.000    0.000   65.703   65.703 /root/projects/assignment1-basics/cs336_basics/pretokenization_example.py:180(process_parallel)
        1    0.000    0.000   65.431   65.431 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/pool.py:738(__exit__)
        1    0.000    0.000   65.401   65.401 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/pool.py:654(terminate)
       21    0.000    0.000   65.381    3.113 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/util.py:272(__call__)
        1    0.000    0.000   65.380   65.380 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/pool.py:680(_terminate_pool)
        1    0.000    0.000   65.342   65.342 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/pool.py:671(_help_stuff_finish)
        1    0.004    0.004   65.342   65.342 {method 'acquire' of '_multiprocessing.SemLock' objects}
       40    0.000    0.000   65.338    1.633 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/connection.py:202(send)
      3/1    0.000    0.000   65.337   65.337 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/threading.py:1000(_bootstrap)
      3/1    0.000    0.000   65.337   65.337 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/threading.py:1027(_bootstrap_inner)
      3/1    0.000    0.000   65.337   65.337 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/threading.py:983(run)
        1    0.000    0.000   65.337   65.337 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/pool.py:527(_handle_tasks)
       45    0.000    0.000   65.337    1.452 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/connection.py:406(_send_bytes)
       45    0.000    0.000   65.337    1.452 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/connection.py:381(_send)
       45    0.000    0.000   65.337    1.452 {built-in method posix.write}
        1    0.000    0.000   65.336   65.336 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/pool.py:573(_handle_results)



================================================================================
Performance Profile (sorted by internal time - shows most time-consuming functions)
================================================================================
         588197441 function calls (588197136 primitive calls) in 3624.529 seconds

   Ordered by: internal time
   List reduced from 511 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1 3091.468 3091.468 3557.664 3557.664 /root/projects/assignment1-basics/cs336_basics/train_bpe.py:51(merge)
583997919/583997879  455.537    0.000  455.537    0.000 {built-in method builtins.len}
  126/122   58.632    0.465   58.633    0.481 {built-in method posix.read}
      301    6.303    0.021    6.303    0.021 {method 'poll' of 'select.poll' objects}
  1369385    5.345    0.000    5.345    0.000 {method 'append' of 'list' objects}
   746269    3.476    0.000    3.476    0.000 {built-in method _heapq.heappop}
  1022782    1.801    0.000    1.801    0.000 {built-in method _heapq.heappush}
        1    0.929    0.929 3624.528 3624.528 /root/projects/assignment1-basics/cs336_basics/train_bpe.py:233(bpe_tokenizer)
       23    0.341    0.015    0.341    0.015 {built-in method _pickle.loads}
       21    0.204    0.010    0.267    0.013 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/collections/__init__.py:673(update)
   495038    0.131    0.000    0.131    0.000 /root/projects/assignment1-basics/cs336_basics/train_bpe.py:253(<genexpr>)
    59935    0.066    0.000    0.066    0.000 {method 'encode' of 'str' objects}
   424682    0.062    0.000    0.062    0.000 {method 'get' of 'dict' objects}
     3213    0.047    0.000    0.391    0.000 {built-in method posix.waitpid}
        1    0.034    0.034   65.737   65.737 /root/projects/assignment1-basics/cs336_basics/pretokenization_example.py:42(wrapper)
     9772    0.033    0.000    0.033    0.000 {method 'items' of 'dict' objects}
     3430    0.008    0.000    0.025    0.000 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/selectors.py:340(register)
      104    0.008    0.000    0.008    0.000 {built-in method builtins.print}
       37    0.007    0.000    0.007    0.000 {method 'read' of '_io.BufferedReader' objects}
     3430    0.007    0.000    0.014    0.000 /root/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/selectors.py:238(register)



================================================================================
Training Summary
================================================================================
Time taken: 1 hour(s), 0 minute(s), 24.61 seconds (3624.61 seconds total)
Memory usage: 5.40 MB (peak: 83.62 MB)
================================================================================

Saved vocab to: /root/projects/assignment1-basics/data/tinystories-train_vocab.json
Saved merges to: /root/projects/assignment1-basics/data/tinystories-train_merges.txt

================================================================================
Longest Token Analysis
================================================================================
Longest token (serialized string): 'Ġaccomplishment'
Token length: 15 characters (in serialized form)
Token ID: 7179
Token bytes: b' accomplishment'
Token bytes length: 15 bytes
Decoded as UTF-8: ' accomplishment'
Does it make sense? Yes - contains readable text
================================================================================

